# Python Apprentice Lecture 3: 미로 탐색과 스택 (Maze Pathfinding with Stack)

이번 단원에서는 **스택을 이용한 미로 탐색**을 배운다.

Lecture 2에서 배운 DFS(깊이 우선 탐색)를 실제 문제에 적용하여 미로에서 출구를 찾는 알고리즘을 구현한다.

## 학습 목표
1. 2D 배열로 미로를 표현하는 방법 이해
2. 좌표 시스템과 방향 벡터 활용
3. 스택을 이용한 DFS로 미로 탐색 구현
4. 경로 추적과 시각화
5. 미로 탐색의 특성과 한계 파악

## 1. 미로 표현과 기본 개념

### 미로의 2D 배열 표현

미로를 2차원 배열로 표현하는 방법:

- **0**: 길 (이동 가능)
- **1**: 벽 (이동 불가)
- **S** 또는 시작점: (0, 0)
- **E** 또는 목표점: (rows-1, cols-1)

```
시각적 표현:     배열 표현:
S . . . .       [[0, 0, 0, 0, 0],
# # . # .        [1, 1, 0, 1, 0],
. . . # .        [0, 0, 0, 1, 0],
# # . . .        [1, 1, 0, 0, 0],
. . . # E        [0, 0, 0, 1, 0]]
```

In [ ]:
maze = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 0, 1, 0]
]

def print_maze(maze, path=None, start=(0,0), end=None):
    if end is None:
        end = (len(maze)-1, len(maze[0])-1)
    
    path_set = set(path) if path else set()
    
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if (i, j) == start:
                print('S', end=' ')
            elif (i, j) == end:
                print('E', end=' ')
            elif (i, j) in path_set:
                print('*', end=' ')
            elif cell == 1:
                print('#', end=' ')
            else:
                print('.', end=' ')
        print()

print("기본 미로:")
print_maze(maze)

### 좌표 시스템과 방향 벡터

미로에서 이동하려면 **4방향 이동**을 정의해야 한다:

```
    상 (0, -1)
      ↑
좌(-1,0) ← 현재 → 우(1, 0)
      ↓
    하 (0, 1)
```

**방향 벡터**: `[(행 변화량, 열 변화량), ...]`

In [ ]:
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def get_neighbors(maze, row, col):
    rows, cols = len(maze), len(maze[0])
    valid_neighbors = []
    
    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < rows and 0 <= new_col < cols:
            if maze[new_row][new_col] == 0:
                valid_neighbors.append((new_row, new_col))
    
    return valid_neighbors

print("(2, 2)에서 이동 가능한 위치:")
neighbors = get_neighbors(maze, 2, 2)
print(neighbors)

### 경계 조건과 유효성 검사

미로 탐색에서 꼭 확인해야 할 조건들:

In [ ]:
def is_valid_move(maze, row, col):
    rows, cols = len(maze), len(maze[0])
    
    if not (0 <= row < rows and 0 <= col < cols):
        return False
    
    if maze[row][col] == 1:
        return False
    
    return True

test_positions = [(0, 0), (0, 1), (2, 2), (-1, 0), (5, 5)]

print("위치 유효성 검사:")
for pos in test_positions:
    row, col = pos
    valid = is_valid_move(maze, row, col)
    print(f"  {pos}: {'유효' if valid else '무효'}")

## 2. 단순 경로 찾기

이제 스택을 이용한 DFS로 미로에서 출구를 찾아보자. 

### 알고리즘 개요

1. **초기화**: 시작점을 스택에 추가, visited 배열 생성
2. **반복**:
   - 스택에서 현재 위치 pop
   - 이미 방문했으면 skip
   - 목표에 도달했으면 성공
   - 현재 위치를 방문 처리
   - 인접한 유효한 위치들을 스택에 추가
3. **종료**: 스택이 비면 경로 없음

Lecture 1의 Stack 클래스를 재사용하자:

In [ ]:
class Stack:
    def __init__(self):
        self.items = []
    
    def push(self, item):
        self.items.append(item)
    
    def pop(self):
        if self.is_empty():
            raise IndexError("Stack is empty")
        return self.items.pop()
    
    def is_empty(self):
        return len(self.items) == 0
    
    def __str__(self):
        return f"Stack: {self.items}"

In [ ]:
def has_path_to_exit(maze, start=(0, 0), end=None):
    if end is None:
        end = (len(maze)-1, len(maze[0])-1)
    
    rows, cols = len(maze), len(maze[0])
    visited = [[False] * cols for _ in range(rows)]
    stack = Stack()
    
    if maze[start[0]][start[1]] == 1:
        return False
    
    stack.push(start)
    
    while not stack.is_empty():
        current = stack.pop()
        row, col = current
        
        if visited[row][col]:
            continue
        
        visited[row][col] = True
        
        if current == end:
            return True
        
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            
            if (is_valid_move(maze, new_row, new_col) and 
                not visited[new_row][new_col]):
                stack.push((new_row, new_col))
    
    return False

result = has_path_to_exit(maze)
print(f"경로 존재: {result}")

In [ ]:
def maze_to_graph(maze):
    rows, cols = len(maze), len(maze[0])
    graph = {}
    
    for i in range(rows):
        for j in range(cols):
            if maze[i][j] == 0:
                pos = (i, j)
                graph[pos] = []
                
                for dr, dc in directions:
                    new_row, new_col = i + dr, j + dc
                    if is_valid_move(maze, new_row, new_col):
                        graph[pos].append((new_row, new_col))
    
    return graph

graph = maze_to_graph(maze)
print("미로 그래프:")
for pos, neighbors in graph.items():
    print(f"{pos}: {neighbors}")